In [1]:
import numpy as np
from scipy.integrate import quad
from datetime import datetime as dt
from datetime import timedelta
import yfinance as yf
from scipy.optimize import minimize
from scipy.stats import norm
import matplotlib.pyplot as plt
from scipy.stats import skew

INPUT PARAMETERS DEFINITION

In [2]:
#S0 INITIAL SPOT PRICE
start = dt(2010,1,1)
end = dt.today()

sp500_ts = yf.download('^GSPC', start = start, end=dt.today())
sp500_ts = sp500_ts['Close']

price_today = np.round(sp500_ts.tail(1)[0])

print('S&P500 price:',price_today)

[*********************100%%**********************]  1 of 1 completed
S&P500 price: 4532.0


C:\Users\e.ceglia\AppData\Local\Temp\ipykernel_7276\1473369541.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  price_today = np.round(sp500_ts.tail(1)[0])


In [3]:
#r RISK FREE RATE
r = 0.0352

In [4]:
#kappa REVERSE MEAN COMPUTED USING AUTOCORRELATION METHOD
sp500_logret = np.log(sp500_ts/sp500_ts.shift(1)).dropna()
autocorr = sp500_logret.autocorr()
reverse_mean = np.round(np.log(-autocorr), 2)
print("Speed of reversion to the mean:",reverse_mean)

Speed of reversion to the mean: -2.1


In [5]:
#sigma VOLATILITY OF VARIANCE
#vix = yf.download("^VIX", start = start, end=dt.today())
#vix = np.round(vix['Close'].tail(1)[0])

vix_hist = yf.download("^VIX", start = start, end=dt.today())
vix_hist = vix_hist['Close']

vix_ret = np.log(vix_hist/vix_hist.shift(1)).dropna()

sigma = np.round(np.std(vix_ret), 2)
print('Volatility of variance:',sigma)

[*********************100%%**********************]  1 of 1 completed
Volatility of variance: 0.08


In [8]:
#Rho CORRELATION BETWEEN UNDERLYING AND HIS VARIANCE
corr_coef = np.round(np.corrcoef(sp500_logret.values, vix_ret.values), 2)
print(corr_coef[0,1])

-0.75


HESTON MODEL PARAMETER DEFINITION

In [9]:
S0 = price_today    # initial spot price
V0 = 0.05   # initial variance
kappa = reverse_mean   # speed of reversion
theta = 0.06  # long term variance
#sigma already defined
rho = corr_coef[0,1]    # correlation between underlying and his variance

PUT OPTION PARAMETER DEFINITION

In [11]:
K = 4165.0     # Strike price found in Step 1
T = 1/12  # Time to maturity 

MONTE-CARLO SIMULATIONS

In [16]:
# Number of Monte-Carlo simulations
N = 1000000

# Monte-Carlo paths for price and varianxce
dt = T/21
days = 21
Z1 = np.random.randn(N, days)  #Random sampling from N(0,1)
Z2 = rho*Z1 + np.sqrt(1-rho**2)*np.random.randn(N, days)  #
S = np.zeros((N, days+1))
V = np.zeros((N, days+1))
S[:,0] = S0  #SPOT PRICE
V[:,0] = V0  #VARIANCE

margin_call = np.zeros(N, dtype=bool)  # initialize array to keep track of margin calls
M = 400 # Margin available

for i in range(days):
    dS = r*S[:,i]*dt + np.sqrt(V[:,i])*S[:,i]*np.sqrt(dt)*Z1[:,i] #Delta prezzo
    dV = kappa*(theta-V[:,i])*dt + sigma*np.sqrt(V[:,i])*np.sqrt(dt)*Z2[:,i] #Delta varianza
    S[:,i+1] = S[:,i] + dS  
    V[:,i+1] = V[:,i] + dV  
    V[:,i+1] = np.maximum(V[:,i+1], 0)  #Confronto la var della montecarlo al tempo t con 0, prendo il massimo o 0, non può essere negativa
    # calculate payoff at this step
    payoff = np.maximum(K - S[:, i+1], 0)
    
    # update margin_call array if payoff is greater than M
    margin_call = np.logical_or(margin_call, payoff > M)

In [18]:
# PAYOFF PUT OPTION
payoff = np.maximum(K-S[:,-1], 0)

# PROBABILITY ITM
prob = np.sum(payoff > 0)/N

print("Probability that option put will be ITM in one month's time::", prob)

# PROBABILITY OF MARGIN CALL
prob_margin_call = np.sum(margin_call) / N

print("Margin call probability:", prob_margin_call)

# for i in range(100):
#     plt.plot(S[i])

Probability that option put will be ITM in one month's time:: 0.094059
Margin call probability: 0.004073
